In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
! pip install -q datasets tqdm

In [3]:
import collections
import random
import os
import time
import json
from PIL import Image
import io
import urllib
import uuid
from concurrent.futures import ThreadPoolExecutor
from functools import partial

import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import matplotlib.pyplot as plt
import tensorflow as tf
from datasets import load_dataset

In [4]:
# Add the relevant ISO code for the language you want to work with.
iso639_3_letter_code = "hau"
# iso639_3_letter_code = "tha"
# iso639_3_letter_code = "kir"

# Download the language specific dataset from HF.
dataset = load_dataset("sil-ai/bloom-captioning", iso639_3_letter_code, 
                       use_auth_token=True, download_mode='force_redownload')

/opt/conda/lib/python3.7/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset bloom_captioning downloaded and prepared to /root/.cache/huggingface/datasets/sil-ai___bloom_captioning/hau/0.0.0/8efe15718b4a50170c9add75b453aec13ec1c5216111d21815428536fe5913ca. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Creating dataframe out of Kyrgyz dataset
import pandas as pd

df_kir_train = pd.DataFrame.from_dict(dataset['train'])
df_kir_test = pd.DataFrame.from_dict(dataset['test'])
df_kir_val = pd.DataFrame.from_dict(dataset['validation'])

In [6]:
iso639_3_letter_code_eng = "eng"

# Download the language specific dataset from HF.
dataset_eng = load_dataset("sil-ai/bloom-captioning", iso639_3_letter_code_eng, 
                       use_auth_token=True, download_mode='force_redownload')

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset bloom_captioning downloaded and prepared to /root/.cache/huggingface/datasets/sil-ai___bloom_captioning/eng/0.0.0/8efe15718b4a50170c9add75b453aec13ec1c5216111d21815428536fe5913ca. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset_eng

DatasetDict({
    test: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story'],
        num_rows: 266
    })
    validation: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story'],
        num_rows: 396
    })
    train: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story'],
        num_rows: 27956
    })
})

In [8]:
!pip install -q googletrans==3.1.0a0
import googletrans
import pandas as pd

# Specifying Destination Language
# lang = 'ky' # For Kyrgyz
lang = 'ha' # For Hausa
# lang = 'th' # For Thai

print(googletrans.LANGUAGES[ lang ])

from googletrans import Translator
translator = Translator()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
pandas-profiling 3.1.0 requires markupsafe~=2.0.1, but you have markupsafe 2.1.1 which is incompatible.
google-api-core 1.33.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 3.19.4 which is incompatible.
gcsfs 2022.5.0 requires fsspec==2022.5.0, but you have fsspec 2022.8.2 which is incompatible.
apache-beam 2.40.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.5.1 which is incompatible.
aiobotocore 2.4.0 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.27.93 which is incompatible.
hausa


In [11]:
# Creating Pandas dataframe out of the Dataset for translation purposes
df_eng_train = pd.DataFrame.from_dict(dataset_eng['train'])
df_eng_val = pd.DataFrame.from_dict(dataset_eng['validation'])

In [12]:
from multiprocessing import Pool, cpu_count

def translate_to_lang(data):
  result = translator.translate(data, dest= lang).text
  return result
# For Training data:
with Pool(processes= cpu_count() ) as p:
  ret = p.map(translate_to_lang, [cap for cap in df_eng_train['caption']])
  df_eng_train['translated_caption'] = ret

# For Validation data:
with Pool(processes= cpu_count() ) as p:
  ret = p.map(translate_to_lang, [cap for cap in df_eng_val['caption']])
  df_eng_val['translated_caption'] = ret

# This takes around 10 mins to run

In [13]:
# Testing sample translation

en_cap = df_eng_train['caption'][1]
ky_cap = df_eng_train['translated_caption'][1]
print(en_cap)
print(ky_cap)

So, Noah’s family had many children. And these children grew up, got married, and had children. People became numerous. Still, they all spoke one same language, the same as Noah.Genesis 11:1
Saboda haka, iyalin Nuhu suna da ’ya’ya da yawa. Su kuma yaran nan sun girma, sun yi aure, sun haihu. Mutane sun yi yawa. Duk da haka, dukansu suna magana da yare ɗaya, kamar yadda Nuhu ya yi.—Farawa 11:1


In [14]:
df_eng_train.head()

,image_id,image_url,caption,story_id,album_id,license,original_bloom_language_tag,index_in_story,translated_caption
0,3904f355-51f4-416a-ae86-820cd7584533,https://bloom-vist.s3.amazonaws.com/103%20Babe...,"God told them, “Bear many children, so that yo...",6039ef03-b7d1-47aa-b2dc-53817e941461,d5fc6b14-8c63-4d95-a26b-3b7769115b10,cc-by-sa,en,0,"Allah ya ce musu, “Ku haifi ’ya’ya da yawa, do..."
1,65d7b9a0-962d-4314-a6d2-f022712eaac2,https://bloom-vist.s3.amazonaws.com/103%20Babe...,"So, Noah’s family had many children. And these...",6039ef03-b7d1-47aa-b2dc-53817e941461,d5fc6b14-8c63-4d95-a26b-3b7769115b10,cc-by-sa,en,1,"Saboda haka, iyalin Nuhu suna da ’ya’ya da yaw..."
2,a5ab567e-c3d8-44ea-87c8-88795a7ed429,https://bloom-vist.s3.amazonaws.com/103%20Babe...,People traveled east [toward the sunrise]. The...,6039ef03-b7d1-47aa-b2dc-53817e941461,d5fc6b14-8c63-4d95-a26b-3b7769115b10,cc-by-sa,en,2,Mutane sun yi tafiya gabas [zuwa fitowar rana]...
3,51aa825d-3771-4b0c-8905-c2e64542bd2f,https://bloom-vist.s3.amazonaws.com/103%20Babe...,"The people said to each other, “We will shape ...",6039ef03-b7d1-47aa-b2dc-53817e941461,d5fc6b14-8c63-4d95-a26b-3b7769115b10,cc-by-sa,en,3,"Jama'a suka ce wa juna, “Za mu yi tubalin laka..."
4,22b17fc0-b92a-4b8f-8f05-2f2fbcf33585,https://bloom-vist.s3.amazonaws.com/103%20Babe...,“We will [must] build a very big city [many st...,6039ef03-b7d1-47aa-b2dc-53817e941461,d5fc6b14-8c63-4d95-a26b-3b7769115b10,cc-by-sa,en,4,"""Dole ne mu gina wani babban birni [gidaje mas..."


In [15]:
col_arrng = ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license','original_bloom_language_tag', 'index_in_story']

# For Traning Data
df_engtokir_train = df_eng_train.drop(['caption'], axis=1)
df_engtokir_train = df_engtokir_train.rename(columns={'translated_caption':'caption'})
df_engtokir_train['original_bloom_language_tag'] = lang
df_engtokir_train = df_engtokir_train[col_arrng]

# For Validation Data
df_engtokir_val = df_eng_val.drop(['caption'], axis=1)
df_engtokir_val = df_engtokir_val.rename(columns={'translated_caption':'caption'})
df_engtokir_val['original_bloom_language_tag'] = lang
df_engtokir_val = df_engtokir_val[col_arrng]

# Merging Traing Data
merge_df_train = pd.concat([df_kir_train, df_engtokir_train], axis=0)
print(merge_df_train.shape)

# Merging Validation Data
merge_df_val = pd.concat([df_kir_val, df_engtokir_val], axis=0)
print(merge_df_val.shape)

(29717, 8)
(448, 8)


In [16]:
merge_df_val.head()

,image_id,image_url,caption,story_id,album_id,license,original_bloom_language_tag,index_in_story
0,e99df6f8-0b17-45e6-b01a-a69539e7b3ec,https://bloom-vist.s3.amazonaws.com/%E0%A4%AF%...,"Bayan kwana biyu sun wuce, Yesu ya cewa almaji...",9e8e9650-9e2a-40c1-ad8a-9b484514df67,72d66eb6-d88a-47ab-a8cb-80932918d88e,cc-by-nc,ha,0
1,6c70f73d-0758-4b55-a755-bbe977097fa0,https://bloom-vist.s3.amazonaws.com/%E0%A4%AF%...,"Almajiran Yesu suka amsa suka ce, “Malam, in L...",9e8e9650-9e2a-40c1-ad8a-9b484514df67,72d66eb6-d88a-47ab-a8cb-80932918d88e,cc-by-nc,ha,1
2,6b689bfd-9e00-40ec-aa74-6a7c09d39c3a,https://bloom-vist.s3.amazonaws.com/%E0%A4%AF%...,"Da Yesu ya iso garin Li’azaru, Li’azaru ya yi ...",9e8e9650-9e2a-40c1-ad8a-9b484514df67,72d66eb6-d88a-47ab-a8cb-80932918d88e,cc-by-nc,ha,2
3,47d00f35-50b3-4b19-ad73-90da313111fa,https://bloom-vist.s3.amazonaws.com/%E0%A4%AF%...,"Yesu ya amsa ya ce, “Ni ne tashin mattatu da k...",9e8e9650-9e2a-40c1-ad8a-9b484514df67,72d66eb6-d88a-47ab-a8cb-80932918d88e,cc-by-nc,ha,3
4,a28a2eeb-3f43-461c-8de7-bcd2afe5dc43,https://bloom-vist.s3.amazonaws.com/%E0%A4%AF%...,Sai Maryamu ta zo. Ta faɗi a ƙafafuwan Yesu ta...,9e8e9650-9e2a-40c1-ad8a-9b484514df67,72d66eb6-d88a-47ab-a8cb-80932918d88e,cc-by-nc,ha,4


In [17]:
from datasets import Dataset, DatasetDict
traindf = Dataset.from_pandas(merge_df_train)
testdf = Dataset.from_pandas(df_kir_test)
valdf = Dataset.from_pandas(merge_df_val)

df = DatasetDict()

df['test'] = testdf
df['validation'] = valdf
df['train'] = traindf

In [18]:
dataset = df
dataset

DatasetDict({
    test: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story'],
        num_rows: 52
    })
    validation: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story', '__index_level_0__'],
        num_rows: 448
    })
    train: Dataset({
        features: ['image_id', 'image_url', 'caption', 'story_id', 'album_id', 'license', 'original_bloom_language_tag', 'index_in_story', '__index_level_0__'],
        num_rows: 29717
    })
})

In [27]:
! rm -rf images
! mkdir images

USER_AGENT = get_datasets_user_agent()

def fetch_single_image(image_url, timeout=None, retries=0):
    request = urllib.request.Request(
        image_url,
        data=None,
        headers={"user-agent": USER_AGENT},
    )
    with urllib.request.urlopen(request, timeout=timeout) as req:
        if 'png' in image_url:
          png = Image.open(io.BytesIO(req.read())).convert('RGBA')
          png.load() # required for png.split()
          background = Image.new("RGB", png.size, (255, 255, 255))
          background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
          image_id = str(uuid.uuid4())
          image_path = "images/" + image_id + ".jpg"
          background.save(image_path, 'JPEG', quality=80)
        else:
          image = Image.open(io.BytesIO(req.read()))
          image_id = str(uuid.uuid4())
          image_path = "images/" + image_id + ".jpg"
          image.save(image_path)
    return image_path

def fetch_images(batch, num_threads, timeout=None, retries=3):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image_path"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch

num_threads = 20
dataset = dataset.map(fetch_images, batched=True, batch_size=2000, fn_kwargs={"num_threads": num_threads})

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

KeyboardInterrupt: 

In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(299, 299)(img)
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
# Get unique train images
encode_train = sorted(set(dataset['train']['image_path']))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

# Get unique test images
encode_test = sorted(set(dataset['test']['image_path']))

# Feel free to change batch_size according to your system configuration
image_dataset_test = tf.data.Dataset.from_tensor_slices(encode_test)
image_dataset_test = image_dataset_test.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset_test):
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

In [ ]:
# Add some special tokens and clean up new line characters.
train_captions = [f"<start> {x} <end>" for x in dataset['train']['caption']]
train_captions = [x.replace('\n', ' ') for x in train_captions]
test_captions = [f"<start> {x} <end>" for x in dataset['test']['caption']]
test_captions = [x.replace('\n', ' ') for x in test_captions]

In [ ]:
caption_dataset = tf.data.Dataset.from_tensor_slices(train_captions)

# We will override the default standardization of TextVectorization to preserve
# "<>" characters, so we preserve the tokens for the <start> and <end>.
def standardize(inputs):
  inputs = tf.strings.lower(inputs)
  return tf.strings.regex_replace(inputs,
                                  r"!\"#$%&\(\)\*\+.,-/:;=?@\[\\\]^_`{|}~", "")

# Max word count for a caption.
max_length = 25
# Use the top 5000 words for a vocabulary.
vocabulary_size = 5000
tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=vocabulary_size,
    standardize=standardize,
    output_sequence_length=max_length)
# Learn the vocabulary from the caption data.
tokenizer.adapt(caption_dataset)

# Create the tokenized vectors
cap_vector = caption_dataset.map(lambda x: tokenizer(x))

# Create mappings for words to indices and indicies to words.
word_to_index = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary())
index_to_word = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary(),
    invert=True)

caption_dataset_test = tf.data.Dataset.from_tensor_slices(test_captions)
cap_vector_test = caption_dataset_test.map(lambda x: tokenizer(x))

In [ ]:
# Create some mas between images, vectors, and captions
img_to_cap_vector = collections.defaultdict(list)
for img, cap in zip(dataset['train']['image_path'], cap_vector):
  img_to_cap_vector[img].append(cap)

img_name_train = []
cap_train = []
for imgt in list(img_to_cap_vector.keys()):
  capt_len = len(img_to_cap_vector[imgt])
  img_name_train.extend([imgt] * capt_len)
  cap_train.extend(img_to_cap_vector[imgt])

img_to_cap_vector_test = collections.defaultdict(list)
for img, cap in zip(dataset['test']['image_path'], cap_vector_test):
  img_to_cap_vector_test[img].append(cap)

img_name_test = []
cap_test = []
for imgv in list(img_to_cap_vector_test.keys()):
  capv_len = len(img_to_cap_vector_test[imgv])
  img_name_test.extend([imgv] * capv_len)
  cap_test.extend(img_to_cap_vector_test[imgv])

In [ ]:
# Feel free to change these parameters according to your system's configuration
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 1024
units = 512
num_steps = len(img_name_train) // BATCH_SIZE

# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [ ]:
# Load the numpy files
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [ ]:
dataset_tf = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset_tf = dataset_tf.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int64]),
          num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and batch
dataset_tf = dataset_tf.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset_tf = dataset_tf.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, 64, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, 64, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, 64, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


In [ ]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))


In [ ]:
# Initialize the encoder and decoder
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, tokenizer.vocabulary_size())

In [ ]:
# Training config.
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

# Loss function to use during training.
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
# Make sure we save checkpoints during training
checkpoint_path = "./checkpoints/new-kir/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [ ]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([word_to_index(tf.constant('<start>'))] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))
  trainable_variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [ ]:
# Adjust this depending on how long you want to train
EPOCHS = 25

# Train our model!
for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset_tf):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

In [ ]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

# Predictions - Vatsal
